In [1]:
"""
Retrain the YOLO model for your own dataset.
"""
import os
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import load_model
from keras.models import Model
from keras.optimizers import adam_v2
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import keras
from scripts.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from scripts.utils import get_random_data
import pickle
import gc

def _main():
    ext = 'mouse'
    annotation_path = '../input/mouse-walking-data/train.txt' # if this changes, the bottlenecks will need to be updated first
    log_dir      = './logs/'
    bott_dir = '../input/bottlenecks/logs/'
    save_path = './'
    weight_path = '../input/trained-weights/'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    classes_path = '../input/model-data/model_data/mouse_classes.txt'
    anchors_path = '../input/model-data/model_data/yolo_anchors_mouse.txt'
    class_names  = get_classes(classes_path)
    num_classes  = len(class_names)
    anchors = get_anchors(anchors_path)

    training_stage = 1# you need to start from 0 if train.txt is changed
    # specify where the training should start (0 is from bottlenecks, 1 is from a draft model)
    
    input_shape = (64,256) # multiple of 32, hw # must match existing bottlenecks if it's being used

    model, bottleneck_model, last_layer_model = create_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='../input/model-data/model_data/yolo.h5') # make sure you know what you freeze

    checkpoint = ModelCheckpoint(log_dir , monitor='val_loss', save_weights_only=True, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

    val_split = 0.1
    
    with open(annotation_path) as f:
        all_lines = f.readlines()
    
    num_val = int(len(all_lines)*val_split)
    num_train = len(all_lines) - num_val

    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    # perform bottleneck training
    if training_stage == -1 :
        print("calculating bottlenecks")
        b = [0 , 1024]
        batch_size=32
        count = 0
        while b[1] < len(all_lines):
            print(count)
            lines = all_lines[b[0]:b[1]]
            bottlenecks = bottleneck_model.predict(data_generator_wrapper(lines, batch_size, input_shape, anchors, num_classes, random=True, verbose=True),steps=(len(lines)//batch_size)+1, max_queue_size=1)
            pickle.dump(bottlenecks[0], open(log_dir + "bottlenecks_"  + str(count) + ext + "0.pickle",'wb'))
            pickle.dump(bottlenecks[1], open(log_dir + "bottlenecks_"  + str(count) + ext + "1.pickle",'wb'))
            pickle.dump(bottlenecks[2], open(log_dir + "bottlenecks_"  + str(count) + ext + "2.pickle",'wb'))
            b[0] += 1024
            b[1] += 1024
            count += 1

    batch_size=32
    
    if training_stage == -1 :
        bott_dir = log_dir

        
    if training_stage <= 0 :
        b = [0 , 1024]
        last_layer_model.compile(optimizer=adam_v2.Adam(0.00005), loss={'yolo_loss': lambda y_true, y_pred: y_pred})
        count = 0
        while b[1] < len(all_lines):
            print(count)
            lines = all_lines[b[0]:b[1]]
            
            b[0] += 1024
            b[1] += 1024
            
            num_val = int(len(lines)*val_split)
            num_train = len(lines) - num_val
            
            dict_bot0 = pickle.load(open(bott_dir + "bottlenecks_" + str(count)+ ext + "0.pickle",'rb'))
            dict_bot1 = pickle.load(open(bott_dir + "bottlenecks_" + str(count)+ ext + "1.pickle",'rb'))
            dict_bot2 = pickle.load(open(bott_dir + "bottlenecks_" + str(count)+ ext + "2.pickle",'rb'))
            dict_bot = {"bot0":dict_bot0,"bot1":dict_bot1,"bot2":dict_bot2}
            
            bottlenecks_train=[dict_bot["bot0"][:num_train], dict_bot["bot1"][:num_train], dict_bot["bot2"][:num_train]]

            bottlenecks_val=[dict_bot["bot0"][num_train:], dict_bot["bot1"][num_train:], dict_bot["bot2"][num_train:]]
            
            print("Training last layers with bottleneck features : " + str(count))
            print('with {} samples, val on {} samples and batch size {}.'.format(num_train, num_val, batch_size) , end = '\r')
            
            
            last_layer_model.fit(bottleneck_generator(lines[:num_train], batch_size, input_shape, anchors, num_classes, bottlenecks_train, random_=True),
                    steps_per_epoch=max(1, num_train//batch_size),
                    validation_data=bottleneck_generator(lines[num_train:], batch_size, input_shape, anchors, num_classes, bottlenecks_val, random_=True),
                    validation_steps=max(1, num_val//batch_size),epochs=20,initial_epoch=0, max_queue_size=1)
            
            del(dict_bot)
            del(dict_bot0)
            del(dict_bot1)
            del(dict_bot2)
            del(bottlenecks_val)
            del(bottlenecks_train)
            del(lines)
            
            gc.collect()
            
            count += 1
            
            if count > 16:
                break
            
        model.save_weights(log_dir + 'trained_weights_stage_0_'  + ext + '.h5')

    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
    if training_stage == 0 :
        weight_path_ = log_dir + 'trained_weights_stage_0_'  + ext + '.h5'
        print("loading from log, Stage 1 " + weight_path_)
        model.load_weights(weight_path_)
        
    else:
        print(model.summary())
        weight_path_ = weight_path + 'trained_weights_stage_0_'  + ext + '.h5'
        print("Starting at Stage 1, loading " + weight_path_)
        model.load_weights(weight_path_)

        
    print('Unfreeze all of the layers.')
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
        
    model.compile(optimizer= adam_v2.Adam(0.0001), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change

    batch_size = 64 # note that more GPU memory is required after unfreezing the body
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))

    model.fit(data_generator_wrapper(all_lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(all_lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=30,
            initial_epoch=0,
            callbacks=[checkpoint, reduce_lr, early_stopping])

    model.save_weights(log_dir + 'trained_weights_final_'  + ext + '.h5')


def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)


def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='../input/model-data/model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    # get output of second last layers and create bottleneck model of it
    out1=model_body.layers[246].output
    out2=model_body.layers[247].output
    out3=model_body.layers[248].output
    bottleneck_model = Model([model_body.input, *y_true], [out1, out2, out3])

    # create last layer model of last layers from yolo model
    in0 = Input(shape=bottleneck_model.output[0].shape[1:].as_list())
    in1 = Input(shape=bottleneck_model.output[1].shape[1:].as_list())
    in2 = Input(shape=bottleneck_model.output[2].shape[1:].as_list())
    last_out0=model_body.layers[249](in0)
    last_out1=model_body.layers[250](in1)
    last_out2=model_body.layers[251](in2)
    model_last=Model(inputs=[in0, in1, in2], outputs=[last_out0, last_out1, last_out2])
    print(anchors)
    print(num_classes)
    model_loss_last =Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_last.output, *y_true])
    last_layer_model = Model([in0,in1,in2, *y_true], model_loss_last)


    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model, bottleneck_model, last_layer_model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes, random=False, verbose=False):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0 and random:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=random)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        if verbose:
            print("Progress: ",i,"/",n)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes, random=False, verbose=False):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes, random, verbose)

def bottleneck_generator(annotation_lines, batch_size, input_shape, anchors, num_classes, bottlenecks,random_=False):
    n = len(annotation_lines)
    print(n)
    i = 0
    while True:
        box_data = []
        b0=np.zeros((batch_size,bottlenecks[0].shape[1],bottlenecks[0].shape[2],bottlenecks[0].shape[3]))
        b1=np.zeros((batch_size,bottlenecks[1].shape[1],bottlenecks[1].shape[2],bottlenecks[1].shape[3]))
        b2=np.zeros((batch_size,bottlenecks[2].shape[1],bottlenecks[2].shape[2],bottlenecks[2].shape[3]))
        for b in range(batch_size):
            _, box = get_random_data(annotation_lines[i], input_shape, random=random_)
            box_data.append(box)
            b0[b]=bottlenecks[0][i]
            b1[b]=bottlenecks[1][i]
            b2[b]=bottlenecks[2][i]
            i = (i+1) % n
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [b0, b1, b2, *y_true], np.zeros(batch_size)

if __name__ == '__main__':
    _main()

2022-01-29 20:41:41.953871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 20:41:42.067394: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 20:41:42.068216: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 20:41:42.069647: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Create YOLOv3 model with 9 anchors and 4 classes.
Load weights ../input/d/raminanush/model-data/model_data/yolo.h5.
Freeze the first 249 layers of total 252 layers.
[[20. 20.]
 [25. 30.]
 [30. 25.]
 [35. 20.]
 [40. 20.]
 [45. 35.]
 [50. 35.]
 [50. 40.]
 [60. 35.]]
4
Tensor("yolo_loss/sub:0", shape=(None, 2, 8, 3, 2), dtype=float32)
Tensor("yolo_loss/strided_slice_26:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("yolo_loss/sub_5:0", shape=(None, 4, 16, 3, 2), dtype=float32)
Tensor("yolo_loss/strided_slice_53:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("yolo_loss/sub_10:0", shape=(None, 8, 32, 3, 2), dtype=float32)
Tensor("yolo_loss/strided_slice_80:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("yolo_loss/sub:0", shape=(None, 2, 8, 3, 2), dtype=float32)
Tensor("yolo_loss/strided_slice_26:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("yolo_loss/sub_5:0", shape=(None, 4, 16, 3, 2), dtype=float32)
Tensor("yolo_loss/strided_slice_53:0", shape

2022-01-29 20:41:55.628584: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
Tensor("model_5/yolo_loss/sub:0", shape=(None, 2, 8, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_26:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/sub_5:0", shape=(None, 4, 16, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_53:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/sub_10:0", shape=(None, 8, 32, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_80:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/sub:0", shape=(None, 2, 8, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_26:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/sub_5:0", shape=(None, 4, 16, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_53:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/sub_10:0", shape=(None, 8, 32, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_80:0", shape=(N

2022-01-29 20:42:04.146169: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:801] layout failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2022-01-29 20:42:05.976345: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


397/397 [==============================] - ETA: 0s - loss: 19.8440Tensor("model_5/yolo_loss/sub:0", shape=(None, 2, 8, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_26:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/sub_5:0", shape=(None, 4, 16, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_53:0", shape=(None, None, None, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/sub_10:0", shape=(None, 8, 32, 3, 2), dtype=float32)
Tensor("model_5/yolo_loss/strided_slice_80:0", shape=(None, None, None, 3, 2), dtype=float32)


2022-01-29 20:47:06.117358: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:801] layout failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2022-01-29 20:47:06.136912: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:801] remapper failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2022-01-29 20:47:06.303694: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:801] remapper failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)


397/397 [==============================] - 340s 816ms/step - loss: 19.8440 - val_loss: 17.0313
Epoch 2/30
397/397 [==============================] - 190s 478ms/step - loss: 17.7516 - val_loss: 15.9155
Epoch 3/30
397/397 [==============================] - 190s 480ms/step - loss: 16.4926 - val_loss: 15.0535
Epoch 4/30
397/397 [==============================] - 191s 482ms/step - loss: 15.4289 - val_loss: 14.5728
Epoch 5/30
397/397 [==============================] - 192s 483ms/step - loss: 14.5192 - val_loss: 13.9960
Epoch 6/30
397/397 [==============================] - 192s 485ms/step - loss: 13.7487 - val_loss: 13.4214
Epoch 7/30
397/397 [==============================] - 193s 487ms/step - loss: 13.1074 - val_loss: 13.6154
Epoch 8/30
397/397 [==============================] - 193s 486ms/step - loss: 12.4904 - val_loss: 13.2413
Epoch 9/30
397/397 [==============================] - 195s 491ms/step - loss: 12.0243 - val_loss: 13.2690
Epoch 10/30
397/397 [==============================] - 19